In [7]:
from PIL import Image

import json
import os
import shutil
import unicodedata
import uuid
import cv2
import mediapipe as mp
from tqdm.auto import tqdm

In [2]:
def create_new_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)           # Removes all the subdirectories!
        print('Directory existed and was cleaned')
        os.makedirs(path)

In [3]:
folder_path_raw = '../dataset/images_raw'
folder_path = '../dataset/images'

In [4]:
def convert_to_jpg(file_path, path_to_save):
    img = Image.open(file_path)
    if img.format == 'PNG': # Conversion of png to jpg
        file_name = os.path.splitext(path_to_save)[0]
        img = img.convert("RGB")
        img.save(file_name + '.jpg', 'JPEG')
    else:
        file_name = os.path.splitext(path_to_save)[0]
        img.save(file_name + '.jpg', 'JPEG')

create_new_folder(folder_path)

for file_name in tqdm(os.listdir(folder_path_raw)):
    if file_name.lower().endswith('.png') or  file_name.lower().endswith('.jpg'):
        file_path = os.path.join(folder_path_raw, file_name)
        path_to_save = os.path.join(folder_path, file_name)
        convert_to_jpg(file_path, path_to_save)

Directory existed and was cleaned


  0%|          | 0/3741 [00:00<?, ?it/s]

100%|██████████| 3741/3741 [02:07<00:00, 29.41it/s]


In [5]:
with os.scandir(folder_path) as entries: # Used scandir instead of listdir to detect Polish characters (e.g ą,ę)
    print(len(list(entries)))

3702


In [21]:
def extract_labels(folder_path):
    labels = {}
    # /home/ant/projects/psl/dataset/images/A35.jpg
    skipped = 0
    created = 0
    labels_folder = '../dataset/labels/'
    images_path = '../dataset/images_renamed'
    
    create_new_folder(labels_folder)
    create_new_folder(images_path)

    # Initialize MediaPipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True)
    mp_drawing = mp.solutions.drawing_utils

    # Searching for files in a folder
    with os.scandir(folder_path) as entries: # Used scandir instead of listdir to detect Polish characters (e.g ą,ę)
        for entry in tqdm(entries):
            if entry.is_file() and entry.name.lower().endswith('.jpg'): # I want to exclude files that are not jpg
                filename = entry.name
                #print(filename)
                label = ''.join([c for c in filename.split('.')[0] if not c.isdigit()])
                
                # Skip file names starting with 'img'
                if filename.lower().startswith('img'):
                    # print(f'skipping starting with img {filename}')
                    skipped += 1
                    continue
                
                # Normalize the filename to ensure proper handling of diacritical marks
                normalized_filename = unicodedata.normalize('NFC', filename)

                # Check if filename contains 'sz', 'cz', 'ch' or 'rz'
                if 'sz' in filename:
                    labels[filename] = 'sz'
                elif 'cz' in filename:
                    labels[filename] = 'cz'
                elif 'ch' in filename:
                    labels[filename] = 'ch'
                elif 'rz' in filename:
                    labels[filename] = 'rz'
                else:    
                    # Retrieving the first letter of the file name    
                    labels[normalized_filename] = label
                
                # Prepare the label dictionary for the current image
                image_labels = {'label': label}
                
                new_file_name = str(uuid.uuid4()).replace('-', '')
                shutil.copyfile(os.path.join(folder_path, filename), os.path.join(images_path, new_file_name + '.jpg'))

                # Process image using MediaPipe: loading image
                image = cv2.imread(os.path.join(folder_path, filename))  # Loading the image from disk
                # Convert the image to RGB format (MediaPipe requires RGB input)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # Processing the image using MediaPipe Hands
                results = hands.process(image_rgb)

                # Get landmarks and add them to the label dictionary
                if results.multi_hand_landmarks:  # Checking if hand landmarks exist in the image processing results
                    for hand_landmarks in results.multi_hand_landmarks:
                        landmarks = hand_landmarks.landmark  # Retrieving the hand landmarks
                        
                        # Creating a dictionary containing the coordinates of the hand landmarks
                        landmark_dict = {f'hand_landmark_{i}': {'x': landmark.x, 'y': landmark.y, 'z': landmark.z} for i, landmark in enumerate(landmarks)}
                        image_labels['hand_landmarks'] = landmark_dict  # Adding the hand landmarks to the label dictionary for the image
                
               # Saving the results to a JSON file
                with open(os.path.join(labels_folder, f'{new_file_name}.json'), 'w', encoding='utf-8') as json_file:
                    json.dump(image_labels, json_file,  indent=4, ensure_ascii=False)
                created += 1
            else:
                skipped += 1  
    print(created, skipped)

# Calling the function
extract_labels(folder_path)

I0000 00:00:1704144742.541699   78925 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1704144742.554311   79699 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.129.03), renderer: NVIDIA GeForce GTX 970/PCIe/SSE2


Directory existed and was cleaned
Directory existed and was cleaned


3702it [02:19, 26.51it/s]

3626 76


In [27]:
def count_files_with_landmarks(labels_folder):
    labeled_with_landmarks_count = 0
    labeled_without_landmarks_count = 0

    # Iterate through files in the labels folder
    for filename in tqdm(os.listdir(labels_folder)):
        if filename.endswith('.json'):  # Check if the file is a JSON file
            with open(os.path.join(labels_folder, filename), 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
                # Check if the JSON file contains the 'landmarks' section
                if 'hand_landmarks' in data:
                    labeled_with_landmarks_count += 1
                else:
                    labeled_without_landmarks_count += 1

    return labeled_with_landmarks_count, labeled_without_landmarks_count

In [28]:
labels_path= '../dataset/labels'
number_of_labeled_files_with_landmarks, number_of_labeled_files_without_landmarks = count_files_with_landmarks(labels_path)
print(f'Number of JSON files with landmarks information: {number_of_labeled_files_with_landmarks}')
print(f'Number of JSON files without landmarks information: {number_of_labeled_files_without_landmarks}')

100%|██████████| 3626/3626 [00:00<00:00, 23475.52it/s]

Number of JSON files with landmarks information: 3262
Number of JSON files without landmarks information: 364
